## A simulation designed to model the movement of sheep being herded by a sheepdog

### Objectives:
- Create an environment for use in deep reinforcement learning
- Create fast running vectorized code using numpy arrays and broadcasting
- Model sheep using flocking behaviours (alignment, cohesion, separation)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
import heapq

class Herd:
    def __init__(self, calculate_reward=False):

        # A very small constant we use to multiply constants by for ease of reading
        self.UNIVERSAL_MODIFIER = 1e-6

        # Initialize constants
        # Size of field
        self.SIZE = 1
        self.GRID_SIZE = (self.SIZE, self.SIZE)

        # Dimensions of the central goal fence area
        self.GOAL_ORIGIN = ((0.4 * self.SIZE), (0.4 * self.SIZE))
        self.GOAL_HEIGHT = 0.2 * self.SIZE
        self.GOAL_WIDTH = 0.2 * self.SIZE

        # Max length of episode
        self.N_STEPS = 1000

        # Number of sheep to be used
        self.N_SHEEP = 3

        # Number of sheep that start inside the goal area
        self.N_SHEEP_HERDED = 2

        # Sheep max speed
        self.MAX_SPEED_SHEEP = 7000 * self.UNIVERSAL_MODIFIER

        # Dog max speed
        self.MAX_SPEED_DOG = 12000 * self.UNIVERSAL_MODIFIER

        # Maximum change of speed per time step for dog (takes 6 frames to decelerate to 0 from max speed)
        self.MAX_ACCELERATION_DOG = 2000 * self.UNIVERSAL_MODIFIER

        # Sheep speed decay, slows sheep slightly each step
        self.SPEED_DECAY_SHEEP = 0.94

        # Movement constants
        self.MAX_RANGE_SCARE = 0.4 * self.SIZE
        self.STRENGTH_SCARE = 25 * self.UNIVERSAL_MODIFIER

        self.MAX_RANGE_EDGE = 0.03 * self.SIZE
        self.STRENGTH_EDGE = 30 * self.UNIVERSAL_MODIFIER 

        self.STRENGTH_NOISE = 300 * self.UNIVERSAL_MODIFIER

        self.MAX_RANGE_COHESION = 0.2 * self.SIZE
        self.MIN_RANGE_COHESION = 0.01 * self.SIZE
        self.STRENGTH_COHESION = 30 * self.UNIVERSAL_MODIFIER 

        self.MAX_RANGE_SEPARATION = 0.03 * self.SIZE
        self.MIN_RANGE_SEPARATION = 0.001 * self.SIZE
        self.STRENGTH_SEPARATION = 400 * self.UNIVERSAL_MODIFIER 

        self.MAX_RANGE_ALIGNMENT = 0.3 * self.SIZE
        self.MIN_RANGE_ALIGNMENT = 0.01 * self.SIZE
        self.STRENGTH_ALIGNMENT = 8000 * self.UNIVERSAL_MODIFIER 

        # Initialize variables
        # Initializes an empty array for future sheep and sheepdog positions as an empty n x 2 array
        self.dog_pos_history = np.empty((self.N_STEPS, 2))
        self.sheep_pos_history = np.empty((self.N_STEPS, self.N_SHEEP, 2))

        # Current time step of the simulation
        self.time_step = 0 

        if calculate_reward:
            self.reward_calculator = self.RewardCalculator()
        else:
            self.reward_calculator = None

    def reset(self):
        '''
        Resets the positions and velocities of the sheep and dog to random initial values.

        Returns:
            (tuple of numpy arrays): Observations from the environment.
        '''
        # Initializes sheep positions as random xy coordinates between 0 and size for each sheep
        self.sheep_pos = self.spawn_sheep(self.N_SHEEP, self.N_SHEEP_HERDED)

        # Initializes sheep veocities as random dx dy vectors
        self.sheep_vel = self.MAX_SPEED_SHEEP * np.random.rand(self.N_SHEEP, 2)

        # Initializes a boolean array, True if sheep is inside goal area
        self.sheep_in_goal = np.full((len(self.sheep_pos),), False)

        # Initialize dog position
        self.dog_pos = np.array([self.SIZE/2, self.SIZE/4])

        # Initialize dog velocity
        self.dog_vel = self.MAX_SPEED_DOG * self.create_random_unit_vectors(1)

        # Current step as 1, as 0 is the initial conditions
        self.time_step = 1

        return self.observe()

    def step(self, action):
        '''
        Moves the simulation forward one step taking in an action from the dog (change in velocity).
        Update positions and velocities of the sheep and dog. 
        Check for wall collisions and limit speeds. 
        Store positions for each step.
        '''

        action = self.limit_speed(action, self.MAX_ACCELERATION_DOG)
        # Move the dog first
        # Add acceleration to dog's velocity
        self.dog_vel += action

        # Immediately limit the dog's speed
        self.dog_vel = self.limit_speed(self.dog_vel, self.MAX_SPEED_DOG)

        # Move the dog
        self.dog_pos += self.dog_vel
        
        # Check dog wall collision
        self.dog_pos, self.dog_vel = self.handle_wall_collisions(self.dog_pos, self.dog_vel)
        self.dog_pos, self.dog_vel = self.handle_fence_collisions(self.dog_pos, self.dog_vel)

        # Move sheep
        # Calculate influences on sheep, adding them to sheep velocities
        self.apply_sheep_influences()

        # Limit sheep speed
        self.sheep_vel = self.limit_speed(self.sheep_vel, self.MAX_SPEED_SHEEP)

        # Decay sheep speed
        self.sheep_vel = self.decay_speed(self.sheep_vel, self.SPEED_DECAY_SHEEP)

        # Move sheep just by current velocity
        self.sheep_pos += self.sheep_vel

        # Collisions with outer walls
        self.sheep_pos, self.sheep_vel = self.handle_wall_collisions(self.sheep_pos, self.sheep_vel)

        # Check if any sheep are inside the goal
        self.check_sheep_inside_goal()

        # Write to storage variables
        self.dog_pos_history[self.time_step] = self.dog_pos
        self.sheep_pos_history[self.time_step] = self.sheep_pos
        self.time_step += 1

        return self.calculate_reward(), self.observe(), self.is_done(), self.get_info()

    def apply_sheep_influences(self):
        '''
        Determine the influence on sheep based on:
            Random noise (random noise is added each time step to simulate randomness)
            Fear of the dog (sheep are strongly repelled by the dog)
            Fear of the edges (sheep are repelled away from nearby edges)
            Flocking - Coherence (sheep are attracted to sheep in a medium distance)
            Flocking - Separation (sheep are repelled by sheep in a short distance)
            Flocking - Alignment (sheep move in the same direction as nearby sheep)

        The method calculates influences, sums them up and adds them to each sheep's current velocity.
        '''
        def calculate_noise_influence():
            '''
            Calculate the noise influence on the sheep.

            The method generates a random unit vector for each sheep to represent the random component 
            of the sheep's movement, also known as the "noise" influence.

            Returns:
                np.array: A 2D array representing the noise influence on each sheep.
                Each row corresponds to a single sheep, while the two columns denote the x and y components of the influence, respectively.
            '''
            # Multiply strength of noise coefficient by random unit vectors (Gaussian noise)
            return self.STRENGTH_NOISE * self.create_random_unit_vectors(self.N_SHEEP)

        def calculate_scare_influence():
            '''
            Calculate the scare influence on the sheep.

            The method calculates the distance and direction between each sheep and the dog, and 
            determines the influence the dog has on each sheep based on these values. The closer 
            the dog is to a sheep, the greater the influence.

            Returns:
                np.array: A 2D array representing the scare influence on each sheep.
                Each row corresponds to a single sheep, while the two columns denote the x and y components of the influence, respectively.
            '''
            # Calculate vectors pointing from dog to sheep for each sheep
            unit_vectors, magnitudes = self.cartesian_to_norm_magnitude(self.sheep_pos - self.dog_pos)

            # This vector multiplied by a scale factor for every sheep within range
            influences = np.where(magnitudes < self.MAX_RANGE_SCARE, self.STRENGTH_SCARE * unit_vectors / (magnitudes ** 2), np.array([0, 0]))

            # Sheep within the central fence have no fear
            return influences * (np.logical_not(self.sheep_in_goal)[:, np.newaxis])
        
        def calculate_edge_influence():
            '''
            Calculates the influence of proximity to the edge on each sheep.

            This method first determines the distance from each sheep to each of the four edges of the simulation area.
            Then, it computes the corresponding influence for both x and y directions.
            This influence is then adjusted for direction (positive for left and bottom edges, negative for right and top edges)
            and the total edge influence is calculated by summing influences from both directions.

            Returns:
                np.array: A 2D array representing the edge influence on each sheep.
                Each row corresponds to a single sheep, while the two columns denote the x and y components of the influence, respectively.
            '''

            # Calculate the distance from the sheep to each of the four edges
            # Numpy arrays are of this shape [[[left, right][bottom, top]],[[left,right]...]]
            distances = np.stack([self.sheep_pos, self.SIZE - self.sheep_pos], axis=-1)

            # Compute influence in both x and y directions
            influences = np.where(distances < self.MAX_RANGE_EDGE, self.STRENGTH_EDGE / distances, 0)
                    
            # Correct the sign of the influence (negative for right and top edges)
            influences[..., 1] *= -1

            return np.sum(influences, axis=-1)

        def calculate_cohesion_influence():
            '''
            Calculates the influence of cohesion (attraction to the flock) on each sheep.

            The method first computes the distance and direction from each sheep to every other sheep.
            Then, it calculates the influence of other sheep that are within a certain range.
            The final influence is the mean of these individual influences.

            Returns:
                np.array: A 2D array representing the cohesion influence on each sheep.
                Each row corresponds to a single sheep, while the two columns denote the x and y components of the influence, respectively.
            '''
            # We expand the sheep pos array so we get a new array of shape [[[x1,y1]],[[x2, y2]],[[x3, y3]]]
            # When we subtract with this array, we will use this shape to make a new array of shape
            # [[[x1-x1, y1-y1],[x2-x1, y2-y1], [x3-x1, y3-y1]],
            #  [[x1-x2, y1-y2],[x2-x2, y2-y2], [x3-x2, y3-y2]],
            #  [[x1-x3, y1-y3],[x2-x3, y2-y3], [x3-x3, y3-y3]]]
            # We then get the unit vectors and magnitudes for each of these by using numpy broadcasting on these vector arrays
            unit_vectors, magnitudes = self.cartesian_to_norm_magnitude(self.sheep_pos - np.expand_dims(self.sheep_pos, axis=1))

            # Create vectorized condition
            condition = np.logical_and((magnitudes <= self.MAX_RANGE_COHESION), (magnitudes >= self.MIN_RANGE_COHESION))

            # Calculate influences if they are within range
            influences = np.where(condition, self.STRENGTH_COHESION * unit_vectors / magnitudes, np.array([0, 0]))

            # Return the mean over the second axis
            return influences.mean(axis=1)
        
        def calculate_separation_influence():
            '''
            Calculates the influence of separation (repulsion from nearby sheep) on each sheep.

            The method first computes the distance and direction from each sheep to every other sheep.
            Then, it calculates the negative influence of other sheep that are within a certain range,
            representing a repulsion effect. The final influence is the mean of these individual influences.

            Returns:
                np.array: A 2D array representing the separation influence on each sheep.
                Each row corresponds to a single sheep, while the two columns denote the x and y components of the influence, respectively.
            '''
            # The same method is used to calculate cohesion influence
            unit_vectors, magnitudes = self.cartesian_to_norm_magnitude(self.sheep_pos - np.expand_dims(self.sheep_pos, axis=1))
            condition = np.logical_and((magnitudes <= self.MAX_RANGE_SEPARATION), (magnitudes >= self.MIN_RANGE_SEPARATION))

            # The only difference is that separation is negative
            influences = -np.where(condition, self.STRENGTH_SEPARATION * unit_vectors / magnitudes, np.array([0, 0]))
            return influences.mean(axis=1)
        
        def calculate_alignment_influence():
            '''
            Calculates the influence of alignment (movement in the same direction as nearby sheep) on each sheep.

            This method first computes the distance from each sheep to every other sheep.
            Then, it calculates the influence of other sheep that are within a certain range, multiplied by the velocity of each sheep,
            representing an alignment effect. The final influence is the mean of these individual influences.

            Returns:
                alignment_influence (np.array): A 2D array representing the alignment influence on each sheep.
                Each row corresponds to a single sheep, while the two columns denote the x and y components of the influence, respectively.
            '''
            # The same method is used to begin calculating alignment
            _, magnitudes = self.cartesian_to_norm_magnitude(self.sheep_pos - np.expand_dims(self.sheep_pos, axis=1))
            condition = np.logical_and((magnitudes <= self.MAX_RANGE_ALIGNMENT), (magnitudes >= self.MIN_RANGE_ALIGNMENT))
            
            # Only instead we discard the unit vectors and multiply by the velocities of each sheep
            influences = np.where(condition, self.STRENGTH_ALIGNMENT * self.sheep_vel / magnitudes, np.array([0, 0]))
            return influences.mean(axis=1)
        
        def calculate_fence_influence():
            '''
            Calculates the influence of the fence on each sheep.

            The method first computes the distance from each sheep to the fence in the left, right, and top.
            Then, it calculates the influence of the fence that are within a certain range.
            The final influence is the sum of these individual influences.

            Returns:
                np.array: A 2D array representing the fence influence on each sheep.
                Each row corresponds to a single sheep, while the two columns denote the x and y components of the influence, respectively.
            '''

            # Fence boundaries
            fence_left = self.GOAL_ORIGIN[0]
            fence_right = self.GOAL_ORIGIN[0] + self.GOAL_WIDTH
            fence_top = self.GOAL_ORIGIN[1] + self.GOAL_HEIGHT
            fence_bottom = self.GOAL_ORIGIN[1]

            # Calculate distances to each fence boundary
            dist_to_left = self.sheep_pos[:,0] - fence_left
            dist_to_right = fence_right - self.sheep_pos[:,0]
            dist_to_top = fence_top - self.sheep_pos[:,1]
            dist_to_bottom = self.sheep_pos[:,1] - fence_bottom


            # Find out which sheep are within the effective range of the fence.
            close_to_left = np.logical_and(dist_to_left <= self.MAX_RANGE_EDGE, np.logical_and(self.sheep_pos[:,1] >= self.GOAL_ORIGIN[1], self.sheep_pos[:,1] <= fence_top))
            close_to_right = np.logical_and(dist_to_right <= self.MAX_RANGE_EDGE, np.logical_and(self.sheep_pos[:,1] >= self.GOAL_ORIGIN[1], self.sheep_pos[:,1] <= fence_top))
            close_to_top = np.logical_and(dist_to_top <= self.MAX_RANGE_EDGE, np.logical_and(self.sheep_pos[:,0] >= fence_left, self.sheep_pos[:,0] <= fence_right))
            close_to_bottom = np.logical_and(self.sheep_in_goal, np.logical_and(dist_to_bottom <= self.MAX_RANGE_EDGE, np.logical_and(self.sheep_pos[:,0] >= fence_left, self.sheep_pos[:,0] <= fence_right)))


            # Calculate the influences
            left_influences = np.where(close_to_left, self.STRENGTH_EDGE / dist_to_left, 0)[:, np.newaxis]
            right_influences = -np.where(close_to_right, self.STRENGTH_EDGE / dist_to_right, 0)[:, np.newaxis]
            top_influences = -np.where(close_to_top, self.STRENGTH_EDGE / dist_to_top, 0)[:, np.newaxis]
            bottom_influences = np.where(close_to_bottom, self.STRENGTH_EDGE / dist_to_bottom, 0)[:, np.newaxis]

            # Combine influences.
            x_influences = left_influences + right_influences
            y_influences = top_influences + bottom_influences

            influences = np.hstack([x_influences, y_influences])

            return influences


        self.sheep_vel += calculate_noise_influence() + calculate_edge_influence() + calculate_scare_influence() + \
         calculate_cohesion_influence() + calculate_separation_influence() + calculate_alignment_influence() + calculate_fence_influence()

    def handle_wall_collisions(self, positions, velocities):
        '''
        Check if sheep or dog have hit a boundary (wall). If so, reflect their positions and velocities accordingly.
        
        Args:
            positions (np.array): Positions of the sheep or the dog.
            velocities (np.array): Velocities of the sheep or the dog.

        Returns:
            positions (np.array): Updated positions.
            velocities (np.array): Updated velocities.
        '''
        
        # Find positions that exceed the area boundaries
        reflect = np.where((positions < 0) | (positions > self.SIZE), -1, 1)
        
        # Reflect velocities and positions
        positions, velocities = positions * reflect, velocities * reflect

        # Positions were over the upper/lower boundaries need double the size added back
        positions = np.where(positions < -self.SIZE, 2*self.SIZE + positions, positions)
   
        return positions, velocities
    
    def handle_fence_collisions(self, positions, velocities):
        '''
        Prevent the dog from entering the fenced area.
        If the dog is within the fence boundaries, it calculates the closest boundary,
        reverses the corresponding velocity component, and adds that velocity to the dog's
        current position to ensure it leaves the boundary.
        
        Args:
            positions (np.array): Positions of the dog.
            velocities (np.array): Velocities of the dog.

        Returns:
            positions (np.array): Updated positions.
            velocities (np.array): Updated velocities.
        '''
        
        # Fence boundaries
        fence_left = self.GOAL_ORIGIN[0]
        fence_right = self.GOAL_ORIGIN[0] + self.GOAL_WIDTH
        fence_top = self.GOAL_ORIGIN[1] + self.GOAL_HEIGHT
        fence_bottom = self.GOAL_ORIGIN[1]
        
        # Check if dog is within the fence boundaries
        within_fence = np.logical_and(
            np.logical_and(fence_left <= positions[0], positions[0] <= fence_right),
            np.logical_and(fence_bottom <= positions[1], positions[1] <= fence_top)
        )
        
        if within_fence:
            # Calculate distances to each fence boundary
            dist_to_left = abs(positions[0] - fence_left)
            dist_to_right = abs(fence_right - positions[0])
            dist_to_top = abs(fence_top - positions[1])
            dist_to_bottom = abs(positions[1] - fence_bottom)
            
            # Find the minimum distance
            min_dist = min(dist_to_left, dist_to_right, dist_to_top, dist_to_bottom)
            
            if (min_dist == dist_to_left) or (min_dist == dist_to_right):
                velocities[0] = -velocities[0]*0.75  # Mirror x velocity
            elif (min_dist == dist_to_top) or (min_dist == dist_to_bottom):
                velocities[1] = -velocities[1]*0.75  # Ensure velocity points down

            # Add velocity to current position to move dog out of the boundary
            positions += velocities

        return positions, velocities
    

    def check_sheep_inside_goal(self):
        # Smaller goal boundaries
        smaller_goal_origin = (self.GOAL_ORIGIN[0] * 1.005, self.GOAL_ORIGIN[1] * 1.005)
        smaller_goal_width = self.GOAL_WIDTH * 0.99
        smaller_goal_height = self.GOAL_HEIGHT * 0.99
        
        fence_left = smaller_goal_origin[0]
        fence_right = smaller_goal_origin[0] + smaller_goal_width
        fence_top = smaller_goal_origin[1] + smaller_goal_height
        fence_bottom = smaller_goal_origin[1]

        self.sheep_in_goal = np.logical_and(np.logical_and(self.sheep_pos[:,0] >= fence_left, self.sheep_pos[:,0] <= fence_right),
                                            np.logical_and(self.sheep_pos[:,1] >= fence_bottom, self.sheep_pos[:,1] <= fence_top))
    
    def create_random_unit_vectors(self, n:int):
        '''
        Helper function, creates n many randomly oriented unit vectors.

        Args:
            n (int): The number of unit vectors to generate.
        
        Returns:
            (n x 2 numpy array): An array of randomly oriented unit vectors
        '''
        random_vectors = np.random.normal(size=(n, 2))
        unit_vectors, _ = self.cartesian_to_norm_magnitude(random_vectors)
        return np.squeeze(unit_vectors)
    
    def spawn_outside_goal(self):
        '''
        Returns coordinates that are outside of the goal area.
        '''
        pos = np.random.rand(1, 2) * np.array([0.6, 0.4])

        def f1(x): return x
        def f2(x): return np.flip(x) + np.array([0.6, 0])
        def f3(x): return x + np.array([0.4, 0.6])
        def f4(x): return np.flip(x) + np.array([0, 0.4])

        return self.SIZE * np.random.choice([f1, f2, f3, f4])(pos)

    def spawn_inside_goal(self):
        '''
        Returns coordinates inside the goal area.
        '''
        return self.SIZE * (np.random.rand(1, 2) * 0.2) + np.array([0.4, 0.4])
        
    def spawn_sheep(self, n_sheep, n_inside=0):
        sheep_pos = np.zeros([n_sheep, 2])
        for i in range(n_sheep):
            if i < n_inside:
                sheep_pos[i] = self.spawn_inside_goal()
            else:
                sheep_pos[i] = self.spawn_outside_goal()

        # Changing the order is important so sheep in goal aren't always in the same array indices
        np.random.shuffle(sheep_pos)
        return sheep_pos

    @staticmethod
    def cartesian_to_norm_magnitude(cartesian_coords):
        '''
        Converts Cartesian coordinates to normalized coordinates and calculates the magnitude.

        Args:
            cartesian_coords (np.array): Array containing Cartesian coordinates.

        Returns:
            normalized (np.array): Array containing normalized coordinates.
            magnitudes (np.array): Array containing magnitudes of each vector passed in.

        '''
        # We the axis=-1 means we get the magnitudes of the innermost objects (vectors)
        # We add a very small value to the magnitudes to avoid divide-by-zero issues.
        magnitudes = np.linalg.norm(cartesian_coords, axis=-1, keepdims=True) + 1e-8

        return cartesian_coords / magnitudes, magnitudes

    @staticmethod
    def limit_speed(velocities, max_speed):
        '''
        Limit the speed (magnitude of velocity vectors) to a maximum value.

        Args:
            velocities (np.array): Array containing velocity vectors.
            max_speed (float): Maximum allowable speed.

        Returns:
            np.array: Array containing the velocity vectors, limited to the maximum speed.
        '''
        # Prevents divide by 0 errors for dog
        if (velocities == np.array([0,0])).all():
            return velocities
        
        # Compute the magnitude of each vector
        magnitudes = np.linalg.norm(velocities, axis=-1, keepdims=True)
        
        return velocities * np.minimum(max_speed / magnitudes, 1)
    
    @staticmethod
    def decay_speed(velocities, decay_rate):
        '''
        Reduces the magnitude of by multiplying by a decay rate.
        
        Args:
            velocities (np.array): Array containing velocity vectors.
            decay_rate (float): Rate at which to decay the speed per time step.

        Returns:
            reduced_velocities (np.array): Array containing the velocity vectors, reduced by decay rate.
        '''
        return decay_rate * velocities
    

    def initialize_scatter(self):
        '''
        Initializes the scatter plot for visualization of the movement of the sheep and sheepdog.
        
        Returns:
            scatter_sheep (matplotlib.collections.PathCollection): Scatter plot object for the sheep.
            scatter_dog (matplotlib.collections.PathCollection): Scatter plot object for the dog.
        '''
        color_grass = '#B6E880'
        color_fence = '#555555'

        # Initialize figure and axis
        self.fig, self.ax = plt.subplots(figsize=(7, 7))
        self.ax.set_facecolor(color_grass)
        self.ax.set_xlim([0, self.SIZE])
        self.ax.set_ylim([0, self.SIZE])

        # Scatter plot to represent sheep and sheepdog
        self.scatter_sheep = self.ax.scatter(self.sheep_pos_history[0,:,0],self.sheep_pos_history[0,:,1]  , c='white', edgecolor="gray", linewidth=0.8, alpha=0.8)
        self.scatter_dog = self.ax.scatter(self.dog_pos_history[0, 0], self.dog_pos_history[0,1], s=35, c='white', edgecolor="black", linewidth=0.8)

        fence_rectangle = patches.Rectangle(self.GOAL_ORIGIN, self.GOAL_WIDTH, self.GOAL_HEIGHT, fc='#B6E880', edgecolor=color_fence, linewidth=2.5)

        # Overwrite bottom edge of fence to make it invisible
        bottom_edge = patches.Polygon([(self.GOAL_ORIGIN[0]*1.01, self.GOAL_ORIGIN[1]), ((self.GOAL_ORIGIN[0]+self.GOAL_WIDTH)*0.995, self.GOAL_ORIGIN[1])], closed=False, fill=True, edgecolor=color_grass, facecolor='white', linewidth=3)

        # Add patch and fence to plot
        self.ax.add_patch(fence_rectangle)
        self.ax.add_patch(bottom_edge)

        return self.scatter_sheep, self.scatter_dog
    
    def update_scatter(self, i): 
        '''
        Updates the scatter plot for each frame of the animation.
        
        Args:
            i (int): Current frame number in the animation.

        Returns:
            scatter_sheep (matplotlib.collections.PathCollection): Updated scatter plot object for the sheep.
            scatter_dog (matplotlib.collections.PathCollection): Updated scatter plot object for the dog.
        '''
        self.scatter_sheep.set_offsets(self.sheep_pos_history[i])
        self.scatter_dog.set_offsets(self.dog_pos_history[i])
        return self.scatter_sheep, self.scatter_dog

    def plot_scatter(self):
        '''
        Plots an animated scatter plot, showing the movement of the sheep and sheepdog over time.
        '''
        FRAMERATE = 30

        ani = animation.FuncAnimation(self.fig, self.update_scatter, interval=round(1000/FRAMERATE), frames=self.N_STEPS, blit=True)
        plt.close()

    def observe(self):
        '''
        Gets current observations from the environment.
        Returns the following information contained in a tuple.

        Returns:
            self.dog_pos (np.array): Numpy array of shape [x, y]
            self.dog_vel (np.array): Numpy array of shape [x, y]
            self.sheep_pos (np.array): Numpy array of shape [[x1, y1], [x2, y2]...]
            self.sheep_vel (np.array): Numpy array of shape [[x1, y1], [x2, y2]...]
        '''
        return (self.dog_pos, self.dog_vel, self.sheep_pos, self.sheep_vel)
    
    def calculate_reward(self):
        '''
        Calculates the reward in the current state of the environment.
        If we have a reward calculator object attached, it will return the reward calculated from that.
        Otherwise, we just return 0 as a dummy value.
        '''
        if self.reward_calculator:
            return self.reward_calculator.calc_reward(self.observe())
        else:
            return 0

    def is_done(self):
        '''
        Flag that checks if the environment has finished running.

        Returns:
            bool: True if the environment has finished running, False otherwise.
        '''
        done = self.time_step >= self.N_STEPS or sum(self.sheep_in_goal) == self.N_SHEEP

        return done
    
    def get_info(self):
        '''
        Method to fetch additional information about the current state of the environment.
        
        Returns:
            tuple: Currently returns an empty tuple. 
                (Will be implemented in future to return additional environment information.)
        '''
        return ()
    
    class RewardCalculator():
        def __init__(self):
            # Constants
            self.SIZE = 1000 # Precision of reward prediction (higher takes longer time to compute intial Dijkstra's)
            self.goal_reward = 10
            self.previous_reward = None  # For use with potential rewards

            # Initialize the grid
            grid = np.ones((self.SIZE, self.SIZE))

            # Define the boundaries
            left, right = int(self.SIZE*0.4), int(self.SIZE*0.6)
            bottom, top = int(self.SIZE*0.4), int(self.SIZE*0.6)

            # Set the fence boundaries as obstacles, value = np.inf
            grid[bottom:top+1, left] = np.inf  # Left fence
            grid[bottom:top+1, right] = np.inf  # Right fence
            grid[top, left:right+1] = np.inf  # Top fence

            # Points on the entrance line
            starts = [(bottom, x) for x in range((left+(1*self.SIZE//25)), (right-(1*self.SIZE//25)+1), 1)]  

            # Perform the Dijkstra algorithm for two sets of start points
            chosen_point = (self.SIZE//35, self.SIZE//2)
            distances1 = self.dijkstra(grid, starts)
            distances2 = self.dijkstra(grid, [chosen_point])

            # Add the two distance grids together
            combined_distances = distances1 + 0.3 * distances2

            # Normalize the combined distances to range [0, 1]
            max_combined_distance = np.max(combined_distances[np.isfinite(combined_distances)])
            combined_distances = combined_distances / max_combined_distance

            # Fix fence values
            combined_distances[bottom:top+1, left] = combined_distances[bottom:top+1, left-1]
            combined_distances[bottom:top+1, right] = combined_distances[bottom:top+1, right+1]
            combined_distances[top, left:right+1] = combined_distances[top + 1, left:right+1]

            # Set the distances inside the square to be very low
            combined_distances[bottom:top, left+1:right] = 0

            self.reward_grid = combined_distances

        def dijkstra(self, grid, starts):
            height, width = grid.shape
            dist = np.full((height, width), np.inf)
            for start in starts:
                dist[start] = 0
            pq = [(0, start) for start in starts]

            while pq:
                d, (x, y) = heapq.heappop(pq)
                if d != dist[x, y]: continue
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    nx, ny = x + dx, y + dy
                    if (0 <= nx < height and 0 <= ny < width and dist[nx, ny] > dist[x, y] + grid[nx, ny]):
                        if np.isinf(grid[nx, ny]): continue  # If cell is an obstacle (part of the fence)
                        dist[nx, ny] = dist[x, y] + grid[nx, ny]
                        heapq.heappush(pq, (dist[nx, ny], (nx, ny)))

            return dist

        def calc_reward(self, obs):
            # This is not an optimized function as we're iterating over each element
            reward_list = []

            # For each sheep
            for pos in obs[2]:
                # Does it fit inside the goal area
                if (pos[0] > 0.4) and (pos[0] < 0.6) and (pos[1] > 0.4) and (pos[1] < 0.6):
                    reward_list.append(self.goal_reward)
                    continue
                
                # Else find its grid location and add the corresponding reward to the reward list
                new_pos = np.floor(pos * self.SIZE).astype(int)
                reward = self.reward_grid[new_pos[0], new_pos[1]]
                reward_list.append(reward)
                
            current_reward = np.mean(reward_list)

            # If this is not our first reward, return the potential reward
            if self.previous_reward:
                potential_reward = current_reward - self.previous_reward
                self.previous_reward = current_reward
                return potential_reward

            # If this is our first reward, set it as our previous reward, then save our current reward
            else:
                self.previous_reward = current_reward
                return 0.



class SimpleAgent:
    def __init__(self, continuous=True):
        self.total_reward = 0
        self.rewards = []

    def discrete_action_wrapper(self, action):
        if action == 0:
            return np.array([0, 0])
        elif action == 1:
            return np.array([-1, 0])
        elif action == 2:
            return np.array([-0.70710678, 0.70710678])
        elif action == 3:
            return np.array([0, 1])
        elif action == 4:
            return np.array([0.70710678, 0.70710678])
        elif action == 5:
            return np.array([1, 0])
        elif action == 6:
            return np.array([0.70710678, -0.70710678])
        elif action == 7:
            return np.array([0, -1])
        elif action == 8:
            return np.array([-0.70710678, -0.70710678])

    def reset(self):
        '''
        Resets total reward for agent.
        '''
        self.total_reward = 0

    def select_action(self, env):
        '''
        Selects an action based off of the current policy.
        The policy for this simple agent is to move in the opposite direction when near the edge of the environment.
        '''
        acceleration = np.array([0, 0])

        dog_pos, _, _, _ = env.observe()

        # 1/boundary is the percentage of the screen size dog will start accelerating away from as it reaches it
        boundary = 20
        if dog_pos[0] > (boundary-1)*env.SIZE/boundary:
            acceleration[0] = -1
        if dog_pos[1] > (boundary-1)*env.SIZE/boundary:
            acceleration[1] = -1
        if dog_pos[0] < env.SIZE/boundary:
            acceleration[0] = 1
        if dog_pos[1] < env.SIZE/boundary:
            acceleration[1] = 1

        return acceleration

    def step(self, env: Herd):
        '''
        Takes a step in the environment.
        '''
        action = self.select_action(env)
        reward, next_obs, done, info = env.step(action)
        self.total_reward += reward
        self.rewards.append(reward)
 

def play_episode(env: Herd, agent):
    '''
    Runs one complete episode in a Herd environment.

    Args:
        env (Herd): The environment in which the agent operates.
        agent (SimpleAgent): The agent that is interacting with the environment.
    '''
    env.reset()
    while env.is_done() == False:
        agent.step(env)
    env.initialize_scatter()
    env.plot_scatter()

agent = SimpleAgent()
env = Herd(calculate_reward=False)

play_episode(env, agent)

![](https://s11.gifyu.com/images/S4b3G.gif)